In [1]:
import os
import glob
import pandas as pd

import numpy as np
import bokeh.io
import bokeh.plotting
import bokeh.palettes
from bokeh.transform import jitter
import seaborn as sns
import matplotlib
from bokeh.models import HoverTool, Range1d
from scipy import stats
import pandas as pd
import math

from bokeh.layouts import row
bokeh.io.output_notebook()

from tkinter import Tk
from tkinter.filedialog import askdirectory

C:\ProgramData\Anaconda3\lib\site-packages\bokeh_catplot\__init__.py:13: DeprecationWarning: bokeh-catplot is deprecated. Use iqplot instead.
  warnings.warn("bokeh-catplot is deprecated. Use iqplot instead.", DeprecationWarning)


Loading BokehJS ...

In [2]:
#Functions
def upgradeCSV(data_dir, app_root_dir, app_data_dir, save_folder, filename_distinct_substring, TechRep):
    extension = 'csv'
    list_csv = []
    for root, dirs, files in os.walk(data_dir, topdown = False):
        for name in files:
            if extension in name:
                if filename_distinct_substring in name:
                    list_csv.append(os.path.join(root, name))
    os.chdir(app_root_dir)
    
    if not os.path.exists(app_data_dir):
        os.mkdir(app_data_dir)
    os.chdir(app_data_dir)
    new_files_dir = save_folder
    if not os.path.exists(new_files_dir):
        os.mkdir(new_files_dir)
    os.chdir(new_files_dir)
    for file in list_csv:
        df_temp = pd.read_csv(file, sep = ',', na_values = '*')
        names = file.rsplit('\\',1)[1].rsplit('_',4)
        df_temp['Strain'] = names[0]
        df_temp['Reporter'] = names[1]
        df_temp['TimePoint'] = names[2].rsplit('h',1)[0]
        df_temp['TeckRep'] = TechRep
        df_temp.to_csv(file.rsplit('\\',1)[1], index = None, header = True)
        
def msd(xdata, ydata):
    r=np.sqrt(xdata**2+ydata**2)
    diff=np.diff(r)
    diff_sq=diff**2
    MSD=np.mean(diff_sq)
    return MSD

def rmsd(xdata, ydata):
    xdata_x0=np.subtract(xdata, xdata[0])
    x=xdata_x0[1:]
    ydata_y0=np.subtract(ydata, ydata[0])
    y=ydata_y0[1:]
    RMSD=np.sqrt(np.mean(x**2+y**2))
    return RMSD

def bootstrap_sampling(my_array, bootstrap_samples, bootstrap_replicates):
    bs_samples=np.zeros((bootstrap_replicates, bootstrap_samples))
    bs_rep=np.arange(bootstrap_replicates)
    for i in bs_rep:
        bs_samples[i, :]=np.random.choice(my_array, bootstrap_samples)
    return bs_samples

def bootstrap_stats(bs_samples):
    samples_shape=np.shape(bs_samples)
    bs_means=np.zeros((samples_shape[0], 1))
    bs_IC_means=np.zeros((1, 2))
    bs_medians=np.zeros((samples_shape[0], 1))
    bs_IC_medians=np.zeros((1, 2))
    for i in np.arange(samples_shape[0]):
        bs_means[i]=np.mean(bs_samples[i, :])
        bs_medians[i]=np.median(bs_samples[i, :])
    bs_IC_means[0, 0]=np.quantile(bs_means, .025)
    bs_IC_means[0, 1]=np.quantile(bs_means, .975)
    bs_IC_medians[0, 0]=np.quantile(bs_medians, .025)
    bs_IC_medians[0, 1]=np.quantile(bs_medians, .975)
    return np.mean(bs_means), np.median(bs_medians), bs_IC_means, bs_IC_medians, stats.sem(bs_medians)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
#set root folder
#data_dir = "D:\\Motility\\"
data_dir = "D:\\Motility_increase\\"
app_root_dir = "C:\\Users\\tala\\Desktop\\git\\PhD_codes\\Mechanosensation\\Python_code\\Motility_increase\\"
#app_root_dir = "C:\\Users\\tala\\Desktop\\Python-Code\\AnalysisMotility\\"
save_folder = "upgraded_csv_files\\"
app_data_dir = "new_data\\"
upgradeCSV(data_dir, app_root_dir, app_data_dir, save_folder, "Track", 1)
upgradeCSV(data_dir, app_root_dir, save_folder, "Spot", 1)
#upgradeCSV(data_dir, app_root_dir, save_folder, "fluo", 1)

os.chdir(app_root_dir + "new_data\\" + save_folder)
extension = 'csv'
list_track_csv = [i for i in glob.glob('*track*.{}'.format(extension))]
print(len(list_track_csv))
list_spot_csv = [i for i in glob.glob('*spot*.{}'.format(extension))]
#list_fluo_csv = [i for i in glob.glob('*fluo*.{}'.format(extension))]

combined_track_csv = pd.concat([pd.read_csv(f) for f in list_track_csv ], sort=False)
combined_spot_csv = pd.concat([pd.read_csv(f) for f in list_spot_csv ], sort=False)
#combined_fluo_csv = pd.concat([pd.read_csv(f) for f in list_fluo_csv ], sort=False)
os.chdir(app_root_dir + "new_data\\")
combined_track_csv.to_csv( "newTrackData.csv", index=False, encoding='utf-8-sig')
combined_spot_csv.to_csv( "newSpotData.csv", index=False, encoding='utf-8-sig')
#combined_fluo_csv.to_csv( "newFluoData.csv", index=False, encoding='utf-8-sig')

24


In [3]:
df_spots = combined_spot_csv.copy()
df_tracks = combined_track_csv.copy()
#df_fluo = combined_fluo_csv.copy()
df_tracks['MSD']=0.0
df_tracks['RMSD']=0.0
df_tracks['PaqaTotalFluorescence']='nan'
df_tracks['rfpTotalFluorescence']='nan'
df_tracks['PaQa_RFP_ratio']='nan'
#df_fluo.head()

NameError: name 'combined_spot_csv' is not defined

In [3]:
app_root_dir = "C:/Users/Iscia/Desktop/git/PhD_codes/Mechanosensation/Python_code/Motility_increase/"
#os.chdir(app_root_dir + "Data\\")
os.chdir(app_root_dir + "new_data\\")

track_duration=40

df_spots = pd.read_csv("newSpotData.csv")
df_tracks_long = pd.read_csv("newTrackData.csv")
df_tracks = df_tracks_long.loc[(df_tracks_long['TRACK_DURATION']>50), :].copy()

#df_fluo = pd.read_csv("newFluoData.csv")
df_tracks['MSD']=0.0
df_tracks['RMSD']=0.0
#df_tracks['PaqaTotalFluorescence']='nan'
#df_tracks['rfpTotalFluorescence']='nan'
#df_tracks['PaQa_RFP_ratio']='nan'
df_tracks.head()

,Label,TRACK_ID,NUMBER_SPOTS,NUMBER_GAPS,LONGEST_GAP,NUMBER_SPLITS,NUMBER_MERGES,NUMBER_COMPLEX,TRACK_DURATION,TRACK_START,...,TRACK_MAX_QUALITY,TRACK_MIN_QUALITY,TRACK_MEDIAN_QUALITY,TRACK_STD_QUALITY,Strain,Reporter,TimePoint,TeckRep,MSD,RMSD
1,Track_6357,1,56,2,4,0,0,0,60,0,...,28.012,5.202,18.547,3.785,cpdA-fliC-,PaQa,0,1,0.0,0.0
7,Track_6363,7,47,6,2,0,0,0,53,0,...,17.951,5.214,13.187,3.770,cpdA-fliC-,PaQa,0,1,0.0,0.0
15,Track_6371,15,62,5,3,0,0,0,69,0,...,16.033,5.214,13.182,2.173,cpdA-fliC-,PaQa,0,1,0.0,0.0
16,Track_6372,16,187,3,3,0,0,0,191,0,...,20.418,5.209,17.974,2.064,cpdA-fliC-,PaQa,0,1,0.0,0.0
18,Track_6374,18,102,3,3,0,0,0,106,0,...,24.018,7.696,17.485,1.852,cpdA-fliC-,PaQa,0,1,0.0,0.0


In [4]:
Strains=list(df_spots.Strain.unique())
TimePoints=list(df_spots.TimePoint.unique())
print(Strains)
print(TimePoints)
print(len(df_tracks))

['cpdA-fliC-', 'cyaB-fliC-', 'fliC-', 'pilG-fliC-', 'pilH-fliC-', 'pilTU-fliC-']
[0, 1, 2, 3]
5528


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
n=0
boot_strain=list()
boot_tp=np.zeros((len(Strains)*len(TimePoints), 1))
print(np.shape(boot_tp))
boot_meanMSD=np.zeros((len(Strains)*len(TimePoints), 1))
boot_medianMSD=np.zeros((len(Strains)*len(TimePoints), 1))
boot_medianMSD_sem=np.zeros((len(Strains)*len(TimePoints), 1))
boot_meanRMSD=np.zeros((len(Strains)*len(TimePoints), 1))
boot_medianRMSD=np.zeros((len(Strains)*len(TimePoints), 1))
boot_medianRMSD_sem=np.zeros((len(Strains)*len(TimePoints), 1))
boot_IC_meanMSD=np.zeros((len(Strains)*len(TimePoints), 2))
boot_IC_medianMSD=np.zeros((len(Strains)*len(TimePoints), 2))
boot_IC_meanRMSD=np.zeros((len(Strains)*len(TimePoints), 2))
boot_IC_medianRMSD=np.zeros((len(Strains)*len(TimePoints), 2))
boot_meanSpeed=np.zeros((len(Strains)*len(TimePoints), 1))
boot_medianSpeed=np.zeros((len(Strains)*len(TimePoints), 1))
boot_medianSpeed_sem=np.zeros((len(Strains)*len(TimePoints), 1))
boot_IC_meanSpeed=np.zeros((len(Strains)*len(TimePoints), 2))
boot_IC_medianSpeed=np.zeros((len(Strains)*len(TimePoints), 2))

for s, strain in enumerate(Strains):
    print(strain)
    for t, tp in enumerate(TimePoints):
        track_IDs=list(df_tracks.TRACK_ID.loc[(df_tracks['Strain']==strain) & (df_tracks['TimePoint']==tp)].unique())
        track_Label=list(df_tracks.Label.loc[(df_tracks['Strain']==strain) & (df_tracks['TimePoint']==tp)].unique())
        print('In Time point '+ str(tp)+' there are '+str(len(track_IDs))+' tracks in SpotData.csv and '+str(len(track_Label))+' in TrackData.csv')
        for track in track_IDs:
            xdata=df_spots.POSITION_X.loc[(df_spots['Strain']==strain) & (df_spots['TimePoint']==tp) & (df_spots['TRACK_ID']==track)].values
            ydata=df_spots.POSITION_Y.loc[(df_spots['Strain']==strain) & (df_spots['TimePoint']==tp) & (df_spots['TRACK_ID']==track)].values
            #paqa=df_fluo.PaqaTotalFluorescence.loc[(df_fluo['Strain']==strain) & (df_fluo['TimePoint']==tp) & (df_fluo['Track_ID']==track)].values
            #rfp=df_fluo.rfpTotalFluorescence.loc[(df_fluo['Strain']==strain) & (df_fluo['TimePoint']==tp) & (df_fluo['Track_ID']==track)].values
            #ratio=df_fluo.PaQa_RFP_ratio.loc[(df_fluo['Strain']==strain) & (df_fluo['TimePoint']==tp) & (df_fluo['Track_ID']==track)].values
            MSD=msd(xdata, ydata)
            RMSD=rmsd(xdata, ydata)
            df_tracks.loc[(df_tracks['Strain']==strain) & (df_tracks['TimePoint']==tp) & (df_tracks['TRACK_ID']==track), 'MSD']=MSD
            df_tracks.loc[(df_tracks['Strain']==strain) & (df_tracks['TimePoint']==tp) & (df_tracks['TRACK_ID']==track), 'RMSD']=RMSD
            #if bool(paqa):
                #df_tracks.loc[(df_tracks['Strain']==strain) & (df_tracks['TimePoint']==tp) & (df_tracks['Label']=='Track_'+str(track)), 'PaqaTotalFluorescence']=paqa
                #df_tracks.loc[(df_tracks['Strain']==strain) & (df_tracks['TimePoint']==tp) & (df_tracks['Label']=='Track_'+str(track)), 'rfpTotalFluorescence']=rfp
                #df_tracks.loc[(df_tracks['Strain']==strain) & (df_tracks['TimePoint']==tp) & (df_tracks['Label']=='Track_'+str(track)), 'PaQa_RFP_ratio']=ratio
        boot_strain.append(strain)
        boot_tp[n]=tp
        tracks_data_MSD=df_tracks.MSD.loc[(df_tracks['Strain']==strain) & (df_tracks['TimePoint']==tp)].values
        bs_tracks_data_MSD=bootstrap_sampling(tracks_data_MSD, len(tracks_data_MSD), 1000)
        [bs_means, bs_medians, IC_means, IC_medians, bs_medians_sem]=bootstrap_stats(bs_tracks_data_MSD)
        boot_meanMSD[n]=bs_means
        boot_IC_meanMSD[n,:]=IC_means
        boot_medianMSD[n]=bs_medians
        boot_medianMSD_sem[n]=bs_medians_sem
        boot_IC_medianMSD[n,:]=IC_medians
        tracks_data_RMSD=df_tracks.RMSD.loc[(df_tracks['Strain']==strain) & (df_tracks['TimePoint']==tp)].values
        bs_tracks_data_RMSD=bootstrap_sampling(tracks_data_RMSD, len(tracks_data_RMSD), 1000)
        [bs_means, bs_medians, IC_means, IC_medians, bs_medians_sem]=bootstrap_stats(bs_tracks_data_RMSD)
        boot_meanRMSD[n]=bs_means
        boot_IC_meanRMSD[n,:]=IC_means
        boot_medianRMSD[n]=bs_medians
        boot_medianRMSD_sem[n]=bs_medians_sem
        boot_IC_medianRMSD[n,:]=IC_medians
        tracks_data_meanSpeed=df_tracks.TRACK_MEAN_SPEED.loc[(df_tracks['Strain']==strain) & (df_tracks['TimePoint']==tp)].values
        bs_tracks_data_meanSpeed=bootstrap_sampling(tracks_data_meanSpeed, len(tracks_data_meanSpeed), 1000)
        [bs_means, bs_medians, IC_means, IC_medians, bs_medians_sem]=bootstrap_stats(bs_tracks_data_meanSpeed)
        boot_meanSpeed[n]=bs_means
        boot_IC_meanSpeed[n,:]=IC_means
        boot_medianSpeed[n]=bs_medians
        boot_medianSpeed_sem[n]=bs_medians_sem
        boot_IC_medianSpeed[n,:]=IC_medians
        n=n+1
n=0
print('Done')

(24, 1)
cpdA-fliC-
In Time point 0 there are 262 tracks in SpotData.csv and 262 in TrackData.csv


KeyboardInterrupt: 

In [7]:
names = ['TimePoint', 'meanMSD','medianMSD','medianMSD_sem', 'meanRMSD', 'medianRMSD', 'medianRMSD_sem', 'meanSpeed', 'medianSpeed', 'medianSpeed_sem']
data = np.concatenate((boot_tp, boot_meanMSD, boot_medianMSD, boot_medianMSD_sem, boot_meanRMSD, boot_medianRMSD, boot_medianRMSD_sem, boot_meanSpeed, boot_medianSpeed, boot_medianSpeed_sem), axis=1)
df_boot_stats=pd.DataFrame(data=np.transpose(data), index=names).T
df_boot_stats['Strain']=boot_strain
df_boot_stats['IC_meanMSD']=list(boot_IC_meanMSD)
df_boot_stats['IC_medianMSD']=list(boot_IC_medianMSD)
df_boot_stats['IC_meanRMSD']=list(boot_IC_meanRMSD)
df_boot_stats['IC_medianRMSD']=list(boot_IC_medianRMSD)
df_boot_stats['IC_meanSpeed']=list(boot_IC_meanSpeed)
df_boot_stats['IC_medianSpeed']=list(boot_IC_medianSpeed)
df_boot_stats['TeckRep']=1
df_boot_stats.head(100)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,TimePoint,meanMSD,medianMSD,medianMSD_sem,meanRMSD,medianRMSD,medianRMSD_sem,meanSpeed,medianSpeed,medianSpeed_sem,Strain,IC_meanMSD,IC_medianMSD,IC_meanRMSD,IC_medianRMSD,IC_meanSpeed,IC_medianSpeed,TeckRep
0,0.0,0.753679,0.721612,0.000927,13.900476,12.817172,0.016002,0.903265,0.9180,0.000341,cpdA-fliC-,"[0.7183433238642306, 0.7910140285163968]","[0.6752293755515414, 0.7828308854678893]","[13.090435254669645, 14.774025678384035]","[11.670554624363117, 13.678640242412811]","[0.8812174485953435, 0.922817938947977]","[0.8999875, 0.937]",1
1,1.0,0.739899,0.760517,0.001914,11.827178,9.776962,0.054684,0.842665,0.9150,0.001425,cpdA-fliC-,"[0.5927714227219086, 0.8801858309838183]","[0.6304468201841652, 0.8456723001639891]","[8.90156486196084, 15.199616210058633]","[7.215000964656899, 12.635252641926131]","[0.7121007142857142, 0.955267142857143]","[0.884, 1.03]",1
2,2.0,0.608139,0.700661,0.002204,9.295533,9.392359,0.041834,0.682488,0.9580,0.000917,cpdA-fliC-,"[0.5011227547714948, 0.7353165762482887]","[0.5425343629429178, 0.7903003228844013]","[7.778103212379353, 10.911708446111845]","[7.120244371852248, 11.616071060986043]","[0.5861905180343144, 0.7737412178055463]","[0.87, 0.995]",1
3,3.0,0.827587,0.827669,0.001232,13.190369,12.921674,0.033375,0.983757,1.0345,0.000597,cpdA-fliC-,"[0.7619779416722103, 0.8947508249341667]","[0.7327831896812429, 0.8628791520853416]","[11.950795504979853, 14.428787236991852]","[10.576180208156332, 14.14664856488736]","[0.9377006048387097, 1.0291947580645162]","[0.9975, 1.0645250000000002]",1
4,0.0,0.162789,0.053545,0.000414,2.281399,1.141570,0.003833,0.273889,0.1520,0.001009,cyaB-fliC-,"[0.12025771837116667, 0.20817585337032135]","[0.03235965965824075, 0.07813227020689703]","[1.8038881890961374, 2.8616875098905608]","[0.9512660109188231, 1.4032113304887457]","[0.22413444531249996, 0.320835019921875]","[0.113, 0.23563749999999986]",1
5,1.0,0.056789,0.013150,0.000110,1.054854,0.736430,0.002412,0.111708,0.0790,0.000382,cyaB-fliC-,"[0.032268138125187494, 0.09528678249822203]","[0.007903564565501977, 0.01995635555922077]","[0.8453023345244158, 1.3006866735792135]","[0.5899228911848242, 0.8768148543956606]","[0.09297141313825497, 0.13214029508440064]","[0.061, 0.109]",1
6,2.0,0.125067,0.075204,0.000355,1.646582,1.136145,0.002399,0.248546,0.1980,0.000606,cyaB-fliC-,"[0.09402417425356335, 0.1667751982613175]","[0.0467053977221181, 0.09124534684649448]","[1.3387452397895028, 2.012996867375613]","[0.9502916920609256, 1.2810823481566667]","[0.2173, 0.2830410852713178]","[0.16699999999999998, 0.23600000000000002]",1
7,3.0,0.126599,0.066934,0.000156,1.980128,1.181552,0.002586,0.278381,0.2220,0.000385,cyaB-fliC-,"[0.10729959436970461, 0.14754580075998694]","[0.05883461056863835, 0.07865741789052742]","[1.741841422949449, 2.2437173379221758]","[1.092791768819647, 1.4222209839798996]","[0.25413995983935744, 0.30367540160642575]","[0.209, 0.249]",1
8,0.0,0.104599,0.044719,0.000113,2.210453,1.569224,0.003041,0.199419,0.1480,0.000267,fliC-,"[0.091544510554723, 0.11993568922363115]","[0.03863144122746282, 0.05263605460981241]","[2.0129627714804195, 2.3976795284518597]","[1.3333624396047918, 1.7405167603396559]","[0.18342112267326735, 0.21535054581188118]","[0.135, 0.16407499999999992]",1
9,1.0,0.230750,0.143040,0.000741,4.279128,3.388210,0.008789,0.424926,0.3820,0.001049,fliC-,"[0.20088163076868235, 0.26601953121003885]","[0.10559251862835893, 0.2196469623684013]","[3.7874238343835698, 4.815357077952445]","[2.7793132510264544, 3.9381583007902727]","[0.38797251344540545, 0.4611492614318649]","[0.32899999999999996, 0.44107499999999994]",1


In [8]:
df_tracks.to_csv( "TrackDataMSD.csv", index=False, encoding='utf-8-sig')
df_boot_stats.to_csv( "BootTrackData.csv", index=False, encoding='utf-8-sig')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
app_root_dir = "C:/Users/Iscia/Desktop/git/PhD_codes/Mechanosensation/Python_code/Motility_increase/"
#os.chdir(app_root_dir + "Data\\")
os.chdir(app_root_dir + "new_data\\")

df_tracks = pd.read_csv("TrackDataMSD.csv")
df_boot_stats = pd.read_csv("BootTrackData.csv")

Strains=list(df_tracks.Strain.unique())
TimePoints=list(df_tracks.TimePoint.unique())
print(Strains)
print(TimePoints)
print(len(df_tracks))

means = df_tracks.groupby(['Strain', 'TimePoint', 'TeckRep'])['RMSD'].mean().to_frame().reset_index()
stds = df_tracks.groupby(['Strain', 'TimePoint', 'TeckRep'])['RMSD'].std().to_frame().reset_index()
medians = df_tracks.groupby(['Strain', 'TimePoint', 'TeckRep'])['RMSD'].median().to_frame().reset_index()
sems = df_tracks.groupby(['Strain', 'TimePoint', 'TeckRep'])['RMSD'].sem().to_frame().reset_index()

meansS = df_tracks.groupby(['Strain', 'TimePoint', 'TeckRep'])['TRACK_MEAN_SPEED'].mean().to_frame().reset_index()
stdsS = df_tracks.groupby(['Strain', 'TimePoint', 'TeckRep'])['TRACK_MEAN_SPEED'].std().to_frame().reset_index()
mediansS = df_tracks.groupby(['Strain', 'TimePoint', 'TeckRep'])['TRACK_MEAN_SPEED'].median().to_frame().reset_index()
semsS = df_tracks.groupby(['Strain', 'TimePoint', 'TeckRep'])['TRACK_MEAN_SPEED'].sem().to_frame().reset_index()

['cpdA-fliC-', 'cyaB-fliC-', 'fliC-', 'pilG-fliC-', 'pilH-fliC-', 'pilTU-fliC-']
[0, 1, 2, 3]
5528


In [4]:
tp=1
a=list(df_boot_stats.loc[(df_boot_stats['Strain'] == 'fliC-') & (df_boot_stats['TimePoint'] == 1), ['IC_medianSpeed']].values[0][0])
b = [tp,tp]
type(b)
type(a)

list

In [8]:
colors = bokeh.palettes.d3['Category10'][10]
parameter='TRACK_MEAN_SPEED'
parameter1='medianSpeed'
parameter2='IC_medianSpeed' #'medianSpeed_sem'
Strains=['fliC-', 'pilTU-fliC-']
pGood = bokeh.plotting.figure(
    width=600, 
    height=600,  
    x_axis_type='linear',
    y_axis_type='linear',
    x_axis_label = 'Time (h)',
    y_axis_label = parameter + ' (µm/s)',
    title="Median tracks speed (bootstrap median, 95% CI)"
)

pGood.xgrid.visible = False
pGood.ygrid.visible = False
pGood.xaxis.minor_tick_line_color = None
pGood.yaxis.minor_tick_line_color = None
pGood.y_range=Range1d(0, 1.1)
pGood.xaxis.ticker = TimePoints

for s, strain in enumerate(Strains):
    for tp in TimePoints:
            b = [tp,tp]
            m=df_boot_stats.loc[(df_boot_stats['Strain'] == strain) & (df_boot_stats['TimePoint'] == tp), [parameter1]].values[0][0]
            sem=list(df_boot_stats.loc[(df_boot_stats['Strain'] == strain) & (df_boot_stats['TimePoint'] == tp), [parameter2]].values[0][0]) #float(semsS[(semsS['Strain'] == strain ) & (semsS['TimePoint'] == tp)][parameter]),
            a=sem
            #if (m-sem < 0):
            #    a = [0, m+sem]
            #else: a = [m-sem, m+sem]
            pGood.line(
                x = b,
                y = a,
                color = 'black',
                alpha=0.8,
                line_width=3
            )
    pGood.line(
        source = df_boot_stats.loc[(df_boot_stats['Strain'] == strain), ['Strain', 'TimePoint', parameter1]],
        x = 'TimePoint',
        y = parameter1,
        line_color = colors[s],
        line_width=3,
        alpha=0.5
    )
    pGood.circle(
        source = df_boot_stats.loc[(df_boot_stats['Strain'] == strain), ['Strain', 'TimePoint', parameter1]],
        x = 'TimePoint',
        y = parameter1,
        line_color = colors[s],
        line_width=2,
        fill_color = 'white',
        alpha=0.5,
        size=10,
        #legend = 'Strain'
    )

Strains=['fliC-', 'pilTU-fliC-', 'pilG-fliC-', 'cyaB-fliC-'] #['fliC-', 'pilG-fliC-', 'pilH-fliC-', 'pilT-pilA-fliC-']
    
pGood2 = bokeh.plotting.figure(
    width=600, 
    height=600,  
    x_axis_type='linear',
    y_axis_type='linear',
    x_axis_label = 'Time (h)',
    y_axis_label = parameter + ' (µm/s)',
    title="Median tracks speed (bootstrap median, 95% CI)"
)

pGood2.xgrid.visible = False
pGood2.ygrid.visible = False
pGood2.xaxis.minor_tick_line_color = None
pGood2.yaxis.minor_tick_line_color = None
pGood2.y_range=Range1d(0, 1.1)
pGood2.xaxis.ticker = TimePoints

for s, strain in enumerate(Strains):
    for tp in TimePoints:
            b = [tp,tp]
            m=df_boot_stats.loc[(df_boot_stats['Strain'] == strain) & (df_boot_stats['TimePoint'] == tp), [parameter1]].values[0][0]
            sem=list(df_boot_stats.loc[(df_boot_stats['Strain'] == strain) & (df_boot_stats['TimePoint'] == tp), [parameter2]].values[0][0]) #float(semsS[(semsS['Strain'] == strain ) & (semsS['TimePoint'] == tp)][parameter]),
            a=sem
            #if (m-sem < 0):
            #    a = [0, m+sem]
            #else: a = [m-sem, m+sem]
            pGood2.line(
                x = b,
                y = a,
                color = 'black',
                alpha=0.8,
                line_width=3
            )
    pGood2.line(
        source = df_boot_stats.loc[(df_boot_stats['Strain'] == strain), ['Strain', 'TimePoint', parameter1]],
        x = 'TimePoint',
        y = parameter1,
        line_color = colors[s],
        line_width=3,
        alpha=0.5
    )
    pGood2.circle(
        source = df_boot_stats.loc[(df_boot_stats['Strain'] == strain), ['Strain', 'TimePoint', parameter1]],
        x = 'TimePoint',
        y = parameter1,
        line_color = colors[s],
        line_width=2,
        fill_color = 'white',
        alpha=0.5,
        size=10,
        #legend = 'Strain'
    )

Strains=['fliC-', 'pilTU-fliC-', 'pilG-fliC-', 'cyaB-fliC-', 'pilH-fliC-', 'cpdA-fliC-']
    
pGood3 = bokeh.plotting.figure(
    width=600, 
    height=600,  
    x_axis_type='linear',
    y_axis_type='linear',
    x_axis_label = 'Time (h)',
    y_axis_label = parameter + ' (µm/s)',
    title="Median tracks speed (bootstrap median, 95% CI)"
)

pGood3.xgrid.visible = False
pGood3.ygrid.visible = False
pGood3.xaxis.minor_tick_line_color = None
pGood3.yaxis.minor_tick_line_color = None
pGood3.y_range=Range1d(0, 1.1)
pGood3.xaxis.ticker = TimePoints

for s, strain in enumerate(Strains):
    for tp in TimePoints:
            b = [tp,tp]
            m=df_boot_stats.loc[(df_boot_stats['Strain'] == strain) & (df_boot_stats['TimePoint'] == tp), [parameter1]].values[0][0]
            sem=list(df_boot_stats.loc[(df_boot_stats['Strain'] == strain) & (df_boot_stats['TimePoint'] == tp), [parameter2]].values[0][0]) #float(semsS[(semsS['Strain'] == strain ) & (semsS['TimePoint'] == tp)][parameter]),
            a=sem
            #if (m-sem < 0):
            #    a = [0, m+sem]
            #else: a = [m-sem, m+sem]
            pGood3.line(
                x = b,
                y = a,
                color = 'black',
                alpha=0.8,
                line_width=3
            )
    pGood3.line(
        source = df_boot_stats.loc[(df_boot_stats['Strain'] == strain), ['Strain', 'TimePoint', parameter1]],
        x = 'TimePoint',
        y = parameter1,
        line_color = colors[s],
        line_width=3,
        alpha=0.5
    )
    pGood3.circle(
        source = df_boot_stats.loc[(df_boot_stats['Strain'] == strain), ['Strain', 'TimePoint', parameter1]],
        x = 'TimePoint',
        y = parameter1,
        line_color = colors[s],
        line_width=2,
        fill_color = 'white',
        alpha=0.5,
        size=10,
        #legend = 'Strain'
    )
    

pGood.output_backend = 'webgl'
pGood2.output_backend = 'webgl'
pGood3.output_backend = 'svg'
bokeh.io.show(bokeh.layouts.row(pGood, pGood2, pGood3))

In [9]:
Strains=list(df_spots.Strain.unique())
timepoints=[0,3]
strain_m=[]*len(Strains)
strain_CI=[]*len(Strains)
m=[None]*len(timepoints)
CI=[None]*len(timepoints)
parameter1='medianSpeed'
parameter2='IC_medianSpeed'
for j, strain in enumerate(Strains):
    for i, tp in enumerate(timepoints):
        m[i]=df_boot_stats.loc[(df_boot_stats['Strain'] == strain) & (df_boot_stats['TimePoint'] == tp), [parameter1]].values[0][0]
        CI[i]=list(df_boot_stats.loc[(df_boot_stats['Strain'] == strain) & (df_boot_stats['TimePoint'] == tp), [parameter2]].values[0][0])
    print(j)
    strain_m[j]=list(m)
    strain_CI[j]=CI

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


NameError: name 'df_spots' is not defined

In [10]:
colors = bokeh.palettes.d3['Category10'][10]
parameter='meanRMSD'
parameter2='IC_meanRMSD'
parameter3='medianRMSD'
parameter4='IC_medianRMSD'


p = bokeh.plotting.figure(
    width=600, 
    height=600,  
    x_axis_type='linear',
    y_axis_type='linear',
    x_axis_label = 'Time (h)',
    y_axis_label = parameter + ' (µm)',
    title="Root mean squared displacement (bootstrap mean, 95% CI)"
)

p1 = bokeh.plotting.figure(
    width=600, 
    height=600,  
    x_axis_type='linear',
    y_axis_type='linear',
    x_axis_label = 'Time (h)',
    y_axis_label = parameter3 + ' (µm)',
    title="Root mean squared displacement (bootstrap median, 95% CI)"
)

In [11]:

for s, strain in enumerate(Strains):
    p.line(
        source = df_boot_stats.loc[(df_boot_stats['Strain'] == strain), ['Strain', 'TimePoint', parameter, parameter2, parameter3, parameter4]],
        x = 'TimePoint',
        y = parameter,
        color = colors[s],
        alpha=0.8,
        #legend = 'Strain'
    )
    p.circle(
        source = df_boot_stats.loc[(df_boot_stats['Strain'] == strain), ['Strain', 'TimePoint', parameter, parameter2, parameter3, parameter4]],
        x = 'TimePoint',
        y = parameter,
        color = colors[s],
        alpha=0.8,
        size=5,
    )

for s, strain in enumerate(Strains):
        for tp in TimePoints:
            b = [tp,tp]
            m=df_boot_stats.loc[(df_boot_stats['Strain'] == strain) & (df_boot_stats['TimePoint'] == tp), [parameter]].values[0][0]
            sem=list(df_boot_stats.loc[(df_boot_stats['Strain'] == strain) & (df_boot_stats['TimePoint'] == tp), [parameter2]].values[0][0]) #float(semsS[(semsS['Strain'] == strain ) & (semsS['TimePoint'] == tp)][parameter]),
            a=sem
            #if (m-sem < 0):
            #    a = [0, m+sem]
            #else: a = [m-sem, m+sem]
            p.line(
                x = b,
                y = a,
                color = colors[s],
                alpha=0.5,
                line_width=3
            )

for s, strain in enumerate(Strains):
    p1.line(
        source = df_boot_stats.loc[(df_boot_stats['Strain'] == strain), ['Strain', 'TimePoint', parameter, parameter2, parameter3, parameter4]],
        x = 'TimePoint',
        y = parameter3,
        color = colors[s],
        alpha=0.8,
        legend = 'Strain'
    )
    p1.circle(
        source = df_boot_stats.loc[(df_boot_stats['Strain'] == strain), ['Strain', 'TimePoint', parameter, parameter2, parameter3, parameter4]],
        x = 'TimePoint',
        y = parameter3,
        color = colors[s],
        alpha=0.8,
        size=5,
    )

for s, strain in enumerate(Strains):
        for tp in TimePoints:
            b = [tp,tp]
            m=df_boot_stats.loc[(df_boot_stats['Strain'] == strain) & (df_boot_stats['TimePoint'] == tp), [parameter3]].values[0][0]
            sem=list(df_boot_stats.loc[(df_boot_stats['Strain'] == strain) & (df_boot_stats['TimePoint'] == tp), [parameter4]].values[0][0]) #float(semsS[(semsS['Strain'] == strain ) & (semsS['TimePoint'] == tp)][parameter]),
            a=sem
            #if (m-sem < 0):
            #    a = [0, m+sem]
            #else: a = [m-sem, m+sem]
            p1.line(
                x = b,
                y = a,
                color = colors[s],
                alpha=0.5,
                line_width=3
            )

p.output_backend = 'svg'
p1.output_backend = 'svg'

bokeh.io.show(bokeh.layouts.row(p, p1))

bokeh.core.validation.check - ERROR - E-1006 (NON_MATCHING_DATA_SOURCES_ON_LEGEND_ITEM_RENDERERS): LegendItem.label is a field, but renderer data sources don't match: LegendItem(id='2756', ...)


In [12]:
colors = bokeh.palettes.d3['Category10'][10]
parameter1='meanRMSD'
parameter2='medianRMSD'
IC1='IC_meanRMSD'
IC2='IC_medianRMSD'
p2 = bokeh.plotting.figure(
    width=600, 
    height=600,  
    x_axis_type='linear',
    y_axis_type='linear',
    x_axis_label = 'Time (h)',
    y_axis_label = parameter1 + ' (µm)',
    title="Root mean squared displacement (bootstrap mean, 95% IC)"
)

p3 = bokeh.plotting.figure(
    width=600, 
    height=600,  
    x_axis_type='linear',
    y_axis_type='linear',
    x_axis_label = 'Time (h)',
    y_axis_label = parameter2 + ' (µm)',
    title="Root mean squared displacement (bootstrap median, 95% IC)"
)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:

for s, strain in enumerate(Strains):
    p2.line(
        source = df_boot_stats.loc[(df_boot_stats['Strain'] == strain), ['Strain', 'TimePoint', parameter1]],
        x = 'TimePoint',
        y = parameter1,
        color = colors[s],
        alpha=0.8,
        legend = 'Strain'
    )
    p2.circle(
        source = df_boot_stats.loc[(df_boot_stats['Strain'] == strain), ['Strain', 'TimePoint', parameter1]],
        x = 'TimePoint',
        y = parameter1,
        color = colors[s],
        alpha=0.8,
        size=5
    )


for s, strain in enumerate(Strains):
    p3.line(
        source = df_boot_stats.loc[(df_boot_stats['Strain'] == strain), ['Strain', 'TimePoint', parameter2]],
        x = 'TimePoint',
        y = parameter2,
        color = colors[s],
        alpha=0.8,
        legend = 'Strain'
    )
    p3.circle(
        source = df_boot_stats.loc[(df_boot_stats['Strain'] == strain), ['Strain', 'TimePoint', parameter2]],
        x = 'TimePoint',
        y = parameter2,
        color = colors[s],
        alpha=0.8,
        size=5
    )  

for s, strain in enumerate(Strains):
        for tp in TimePoints:
            b = [tp,tp]
            m=df_boot_stats.loc[(df_boot_stats['Strain'] == strain) & (df_boot_stats['TimePoint'] == tp), [parameter1]].values[0][0]
            ic=df_boot_stats.loc[(df_boot_stats['Strain'] == strain) & (df_boot_stats['TimePoint'] == tp), [IC1]].values[0][0]
            if (m-ic[0] < 0):
                a = [0, m+ic[1]]
            else: a = [m-ic[0], m+ic[1]]           
            p2.line(
                x = b,
                y = ic,
                color = colors[s],
                alpha=0.5,
                line_width=3
            )
for s, strain in enumerate(Strains):
        for tp in TimePoints:
            b = [tp,tp]
            m=df_boot_stats.loc[(df_boot_stats['Strain'] == strain) & (df_boot_stats['TimePoint'] == tp), [parameter2]].values[0][0]
            ic=df_boot_stats.loc[(df_boot_stats['Strain'] == strain) & (df_boot_stats['TimePoint'] == tp), [IC2]].values[0][0]
            if (m-ic[0] < 0):
                a = [0, m+ic[1]]
            else: a = [m-ic[0], m+ic[1]]            
            p3.line(
                x = b,
                y = ic,
                color = colors[s],
                alpha=0.5,
                line_width=3
            )

p2.output_backend = 'svg'
p3.output_backend = 'svg'

bokeh.io.show(bokeh.layouts.row(p2, p3))

UFuncTypeError: ufunc 'subtract' did not contain a loop with signature matching types (dtype('<U32'), dtype('<U32')) -> dtype('<U32')

In [14]:
colors = bokeh.palettes.d3['Category10'][10]
parameter1='RMSD'
parameter2='PaQa_RFP_ratio'
p5 = bokeh.plotting.figure(
    width=600, 
    height=600,  
    x_axis_type='log',
    y_axis_type='log',
    x_axis_label = parameter1,
    y_axis_label = parameter2,
    title="PaQa/RFP ratio vs RMSD"
)
Strains2=['fliC-', 'pilH-fliC-']
for s, strain in enumerate(Strains):
    p5.circle(
        source = df_tracks.loc[(df_tracks['Strain'] == strain) & (df_tracks['TRACK_START'] == 0), [parameter1, parameter2]],
        x = parameter1,
        y = parameter2,
        color = colors[s],
        alpha=0.8,
        size=5,
        legend=strain
    )

bokeh.io.show(p5)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


KeyError: "Passing list-likes to .loc or [] with any missing labels is no longer supported. The following labels were missing: Index(['PaQa_RFP_ratio'], dtype='object'). See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike"

In [15]:
for strain in Strains:
    X=np.array(df_tracks.loc[(df_tracks['Strain'] == strain) & (df_tracks['TRACK_START'] == 0), [parameter1]].values, dtype=np.float32)
    Y=np.array(df_tracks.loc[(df_tracks['Strain'] == strain) & (df_tracks['TRACK_START'] == 0), [parameter2]].values, dtype=np.float32)
    print(parameter1+' vs '+parameter2+' in '+strain+':')
    [spearman_r, spearman_p]=stats.spearmanr(X, Y)
    print('Spearman correlation = '+str(spearman_r)+', p-value = '+ str(spearman_p))
    [pearson_r, pearson_p]=stats.pearsonr(X, Y)
    print('Pearson correlation = '+str(pearson_r)+', p-value = '+str(pearson_p))
    print('')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


KeyError: "None of [Index(['PaQa_RFP_ratio'], dtype='object')] are in the [columns]"

In [16]:
colors = bokeh.palettes.d3['Category10'][10]
parameter1='TimePoint'
parameter2='PaQa_RFP_ratio'
p6 = bokeh.plotting.figure(
    width=600, 
    height=600,  
    x_axis_type='linear',
    y_axis_type='log',
    x_axis_label = parameter1,
    y_axis_label = parameter2,
    title="PaQa/RFP ratio vs timePoint"
)
Strains2=['fliC-', 'pilH-fliC-']
for s, strain in enumerate(Strains2):
    p6.circle(
        source = df_tracks.loc[(df_tracks['Strain'] == strain) & (df_tracks['TRACK_START'] == 0), [parameter1, parameter2]],
        x = parameter1,
        y = parameter2,
        color = colors[s],
        alpha=0.8,
        size=5,
        legend=strain
    )

bokeh.io.show(p6)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


KeyError: "Passing list-likes to .loc or [] with any missing labels is no longer supported. The following labels were missing: Index(['PaQa_RFP_ratio'], dtype='object'). See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike"